In [1]:
NUM_LABELS = 27
text_max_len = 512
smiles_max_len = 512
batch_size = 16
text_model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
smiles_model_name = "DeepChem/ChemBERTa-77M-MLM"
path_to_data = "../data/classification/smiles/train_augmented"
model_path = "../models/smiles/train_augmented/all/model.pt"

In [2]:
import sys
import os

original_sys_path = sys.path.copy()
try:
    sys.path.insert(0, os.path.abspath(".."))
    from src.modeling_helpers import *
finally:
    sys.path = original_sys_path


def load_trial_data_eval_drug_class(path):

    # File paths for train, validation, and test datasets
    train_path = os.path.join(path, "train.csv")
    val_path = os.path.join(path, "val.csv")
    test_path = os.path.join(path, "test.csv")

    # Columns to be dropped from the datasets
    to_drop = [
        "nctid",
        "group_id",
        "title",
        "intervention_name",
        "intervention_name_drugbank",
        "type",
        "kingdom",
        "sequence",
        "is_placebo",
        "contain_placebo",
    ]

    # Load and process the training data
    train_df = pd.read_csv(train_path).drop(columns=to_drop)

    # Load and process the validation data
    val_df = pd.read_csv(val_path).drop(columns=to_drop)

    # Load and process the test data
    test_df = pd.read_csv(test_path).drop(columns=to_drop)

    return train_df, val_df, test_df

In [3]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Instansiate tokenizers
group_desc_tokenizer = AutoTokenizer.from_pretrained(text_model_name)
eligibility_tokenizer = AutoTokenizer.from_pretrained(text_model_name)
smiles_tokenizer = AutoTokenizer.from_pretrained(smiles_model_name)
smiles_tokenizer.add_tokens(["[PLACEBO]", "[NOSMILES]"])
smiles_tokenizer_len = len(smiles_tokenizer)

model = MultilabelModel(
    text_model_name, smiles_model_name, smiles_tokenizer_len, NUM_LABELS
)

# Load the model state dictionary
model.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode
model.eval()

# If you're using a specific device (e.g., GPU), ensure to move the model to the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

train_df, val_df, test_df = load_trial_data_eval_drug_class(path_to_data)

atc_test_groups = test_df.groupby("atc_anatomical_main_group")

results = {}

for atc_main, group in tqdm(atc_test_groups):

    group_copy = group.copy()
    group_copy.drop("atc_anatomical_main_group", axis=1, inplace=True)
    group_copy.reset_index(drop=True, inplace=True)
    # Test datasets
    test_dataset = CustomDataset(
        group_copy,
        group_desc_tokenizer,
        eligibility_tokenizer,
        smiles_tokenizer,
        text_max_len,
        smiles_max_len,
    )
    test_dataloader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn
    )
    # Run validation (or testing in this context)
    test_metrics = validate_full(model, test_dataloader, device, 0, 0)

    # Unpack your test metrics
    f1_micro, _, _, _, _, overall_BalancedAccuracy, _, _, _, _, _, _ = test_metrics

    # Store metrics in the results dictionary for each atc_main
    results[atc_main] = {
        "Support": len(group_copy),
        "F1 Micro": f1_micro,
        "Balanced Accuracy": overall_BalancedAccuracy,
    }

atc_labels = {
    "[NOATCM]": "[NOATCM]",
    "a": "Alimentary tract and metabolism",
    "b": "Blood and blood forming organs",
    "c": "Cardiovascular system",
    "d": "Dermatologicals",
    "g": "Genito urinary system and sex hormones",
    "h": "Systemic hormonal preparations, excl. sex hormones and insulins",
    "j": "Antiinfectives for systemic use",
    "l": "Antineoplastic and immunomodulating agents",
    "m": "Musculo-skeletal system",
    "n": "Nervous system",
    "p": "Antiparasitic products, insecticides and repellents",
    "r": "Respiratory system",
    "s": "Sensory organs",
    "v": "Various",
}
result_df = pd.DataFrame(results)
result_df.iloc()[1:, :] = round(result_df.iloc()[1:, :] * 100, 2)
result_df.columns = [atc_labels[i] for i in result_df.columns]
warnings.filterwarnings("default")
result_df.T

Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/13 [00:00<?, ?it/s]

Epoch 1/0:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/0:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/0:   0%|          | 0/4 [00:00<?, ?it/s]

...

Epoch 1/0:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/0:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/0:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/0:   0%|          | 0/4 [00:00<?, ?it/s]

,Support,F1 Micro,Balanced Accuracy
[NOATCM],65.0,71.87,81.51
Alimentary tract and metabolism,44.0,68.53,78.67
Blood and blood forming organs,49.0,73.09,76.76
Cardiovascular system,51.0,78.48,84.41
Dermatologicals,16.0,52.75,73.82
Genito urinary system and sex hormones,33.0,62.50,76.80
"Systemic hormonal preparations, excl. sex hormones and insulins",21.0,73.46,79.62
Antiinfectives for systemic use,73.0,63.00,75.91
Antineoplastic and immunomodulating agents,211.0,77.67,79.90
Nervous system,92.0,73.98,82.80
